# Molecular Dynamics Analysis Tutorial: From Trajectories to Phonons

This tutorial explains how to analyze LAMMPS molecular dynamics trajectories to extract phonon properties using Spectral Energy Density (SED) analysis. We'll cover the complete workflow from loading trajectory data to generating phonon dispersion plots, with detailed explanations and implementations of each step.

## Table of Contents
1. Loading Trajectory Data
2. Computing Average Positions
3. Understanding SED Calculation
4. Practical SED Analysis
5. Visualization and Interpretation
6. Complete Examples
7. Tips for Best Results

### Dependencies
```python
import numpy as np
from tqdm import tqdm
from ovito.io import import_file 
import matplotlib.pyplot as plt  
```

## 1. Loading Trajectory Data

This section covers how to read and process molecular dynamics trajectory files from LAMMPS simulations.

### Required Dump File Format

Your LAMMPS dump file must contain the following columns in order:
```
id type x y z vx vy vz
```

Example LAMMPS dump command:
```lammps
dump 1 all custom 100 trajectory.dump id type x y z vx vy vz
```

Example file format:
```
ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
1000
ITEM: BOX BOUNDS pp pp pp
0.0 100.0
0.0 100.0
0.0 100.0
ITEM: ATOMS id type x y z vx vy vz
1 1 23.5 45.2 67.8 0.001 0.002 -0.001
2 1 24.6 44.3 68.9 -0.001 0.001 0.002
...
```

### Complete qdump Implementation

```python
def qdump(filename, timescaling=1, convert=True):
    """
    Load molecular dynamics trajectory data.
    
    Args:
        filename (str): Path to the trajectory file
        timescaling (float): Time step scaling factor
        convert (bool): Whether to save processed data as .npy files
        
    Returns:
        tuple: (positions, velocities, timesteps, atom_types)
    """
    import os
    import numpy as np
    from tqdm import tqdm
    from ovito.io import import_file
    
    # Check for existing processed data
    if os.path.exists(filename + "_at.npy"):
        print("Reading from cached .npy files")
        ts = np.load(filename + "_t.npy")
        pos = np.load(filename + "_p.npy")
        vel = np.load(filename + "_v.npy")
        at = np.load(filename + "_at.npy")
        return pos, vel, ts, at

    print("Processing dump file")
    pipeline = import_file(filename)
    nt = pipeline.source.num_frames  # Total number of timesteps
    
    # Get dimensions from first frame
    data = pipeline.compute(0)
    na, nxyz = np.shape(data.particles.positions.array)
    
    # Initialize arrays
    pos = np.zeros((nt, na, 3))
    vel = np.zeros((nt, na, 3))
    at = np.zeros((nt, na), dtype=int)
    ts = np.arange(nt) * timescaling
    
    # Read each frame
    for n in tqdm(range(nt)):
        data = pipeline.compute(n)
        pos[n, :, :] = data.particles.position.array
        vel[n, :, :] = data.particles.velocities.array
        at[n, :] = data.particles.particle_types.array

    # Cache processed data if requested
    if convert:
        np.save(filename + "_t.npy", ts)
        np.save(filename + "_p.npy", pos)
        np.save(filename + "_v.npy", vel)
        np.save(filename + "_at.npy", at)

    return pos, vel, ts, at
```

Basic usage:
```python
# Load a trajectory file
positions, velocities, timesteps, atom_types = qdump("simulation.dump")
```

Features:
- Automatic caching to `.npy` files for faster reloading
- Progress bar for large trajectories

## 2. Computing Average Positions

This section explains how to calculate time-averaged atomic positions and displacement vectors, which are crucial for SED analysis

### Complete avgPos Implementation

```python
def avgPos(pos, sx, sy, sz, alpha=90, beta=90, gamma=90):
    """
    Calculate average positions and displacements.
    
    Args:
        pos (numpy.ndarray): Position array [timesteps, atoms, xyz]
        sx, sy, sz (float): System dimensions
        alpha, beta, gamma (float): Unit cell angles
        
    Returns:
        tuple: (average_positions, displacements)
    """
    import numpy as np
    from tqdm import tqdm
    
    nS, nA, na = np.shape(pos)
    displacements = np.zeros((nS, nA, na))
    
    # Calculate displacements from initial positions
    for t in tqdm(range(nS)):
        displacements[t,:,:] = dxyz(pos[0,:,:], 
                                  pos[t,:,:],
                                  sx, sy, sz,
                                  alpha, beta, gamma)
    
    # Average position = initial + mean displacement
    avg_pos = pos[0,:,:] + np.mean(displacements[:,:,:], axis=0)
    
    # Displacements from average position
    disp_from_avg = displacements - np.mean(displacements[:,:,:], axis=0)
    
    return avg_pos, disp_from_avg

def getWrapAndRange(size, axis):
    """
    Determine wrapping parameters for periodic boundaries.
    
    Args:
        size (float): System size in given direction
        axis (int): Axis index (0=x, 1=y, 2=z)
        
    Returns:
        tuple: (wrap_vector, range_list)
    """
    import numpy as np
    
    wrap = np.zeros(3)
    if size is None:
        wrap[axis] = 1
        r = [0]
    else:
        wrap[axis] = size
        r = [-1, 0, 1]
    return wrap, r

def dxyz(pos1, pos2, sx, sy, sz, alpha=90, beta=90, gamma=90):
    """
    Calculate minimum distances between positions considering periodicity.
    
    Args:
        pos1, pos2 (numpy.ndarray): Position arrays
        sx, sy, sz (float): System dimensions
        alpha, beta, gamma (float): Unit cell angles
        
    Returns:
        numpy.ndarray: Minimum distances
    """
    import numpy as np
    
    dxyz_0 = pos2 - pos1
    
    # Get wrapping parameters for each direction
    wx, i_range = getWrapAndRange(sx, 0)
    wy, j_range = getWrapAndRange(sy, 1)
    wz, k_range = getWrapAndRange(sz, 2)
    
    # Check all periodic images
    for i in i_range:
        for j in j_range:
            for k in k_range:
                shift_xyz = i*wx + j*wy + k*wz
                
                # Handle non-orthogonal cells
                if gamma != 90:
                    skew = np.eye(3)
                    skew[0,1] = -np.sin(gamma*np.pi/180 - np.pi/2)
                    skew[1,1] = np.cos(gamma*np.pi/180 - np.pi/2)
                    shift_xyz = np.matmul(skew, shift_xyz)
                
                dxyz_w = pos2 + shift_xyz - pos1
                dxyz_0 = absMin(dxyz_0, dxyz_w)
    
    return dxyz_0

def absMin(dxyz_a, dxyz_b):
    """
    Select minimum absolute distances between two sets.
    
    Args:
        dxyz_a, dxyz_b (numpy.ndarray): Distance arrays to compare
        
    Returns:
        numpy.ndarray: Array with minimum absolute distances
    """
    import numpy as np
    
    abs1 = np.absolute(dxyz_a)
    abs2 = np.absolute(dxyz_b)
    minabs = np.minimum(abs1, abs2)
    
    keeps = np.zeros((len(dxyz_a), 3))
    keeps[abs1 == minabs] = dxyz_a[abs1 == minabs]
    keeps[abs2 == minabs] = dxyz_b[abs2 == minabs]
    
    return keeps
```

Basic usage:
```python
# Calculate average positions
avg_positions, displacements = avgPos(positions, 
                                    sx=100.0,  # system size in x 
                                    sy=100.0,  # system size in y
                                    sz=100.0)  # system size in z
```

Notes:
- Automatically handles periodic boundary conditions
- `sx`, `sy`, `sz` should match your simulation box dimensions (get these from the dump file header)
- Non-orthogonal cells supported via optional `alpha`, `beta`, `gamma` parameters

## 3. Understanding SED Calculation

This section provides the mathematical framework behind Spectral Energy Density analysis and its implementation.

### Mathematical Framework

Reference
DOI: 10.1103/PhysRevB.81.081411

### 1. Atomic Displacement Definition
For each atom $j$ at time $t$, we define displacement $u_j(t)$ from equilibrium position $r_j^0$:

$u_j(t) = r_j(t) - r_j^0$

### 2. Velocity Decomposition
The velocity of each atom can be projected onto a chosen direction $\hat{e}$:

$v_j(t) = \dot{u}_j(t) \cdot \hat{e}$

### 3. Phase Factor Calculation
For a given wave vector $\mathbf{k}$, we compute phase factors based on atomic positions:

$\phi_j(\mathbf{k}) = e^{i\mathbf{k}\cdot\mathbf{r}_j^0}$

### 4. Space-Time Fourier Transform
The SED $\Phi(\mathbf{k},\omega)$ is computed as:

$\Phi(\mathbf{k},\omega) = \frac{1}{2\pi N\tau} \left|\sum_{j=1}^N \int_0^\tau v_j(t)\phi_j(\mathbf{k})e^{-i\omega t}dt\right|^2$

where:
- $N$ is the number of atoms
- $\tau$ is the total simulation time
- $\omega$ is the angular frequency

### 5. Discrete Implementation
In practice, we work with discrete time steps and finite systems:

$\Phi(\mathbf{k},\omega) = \frac{1}{2\pi N M} \left|\sum_{j=1}^N \sum_{m=0}^{M-1} v_j(t_m)e^{i\mathbf{k}\cdot\mathbf{r}_j^0}e^{-i\omega t_m}\Delta t\right|^2$

where:
- $M$ is the number of time steps
- $\Delta t$ is the time step size
- $t_m = m\Delta t$

### 6. Implementation Details
The code implements this in two main steps:

a) Spatial transform (for each time step):

$F(t) = \frac{1}{N}\sum_{j=1}^N v_j(t)e^{i\mathbf{k}\cdot\mathbf{r}_j^0}$

b) Temporal transform:

$\tilde{F}(\omega) = \mathcal{F}[F(t)]$

where $\mathcal{F}$ denotes the Fast Fourier Transform (FFT).

## Code Implementation

### 1. Position Projection
```python
# Project atomic positions onto wave vector direction
if isinstance(p_xyz, (int, float)):
    xs = avg[bs, p_xyz]  # Project positions onto chosen direction
else:
    # Handle arbitrary direction vectors
    xs = np.einsum('ij, ij->i', avg[bs, :], direction_vector)
```

This step implements the projection of atomic positions onto the wave vector direction, corresponding to the $\mathbf{r}_j^0$ term in the mathematical formulation.

### 2. Velocity Projection
```python
# Project velocities onto analysis direction
if isinstance(v_xyz, (int, float)):
    vs = velocities[:, bs, v_xyz]  # Select velocity component
else:
    # Handle arbitrary velocity vectors
    vflat = np.reshape(velocities[:, bs, :], (nt*na, 3))
    vs = np.einsum('ij, ij->i', vflat, direction_vector)
    vs = np.reshape(vs, (nt, na))
```

This implements the velocity decomposition $v_j(t)$, projecting atomic velocities onto the chosen analysis direction.

### 3. Phase Factor and Spatial Transform
```python
# Calculate phase factors and perform spatial transform
F = vs[:, :] * np.exp(1j * k * xs[None, :])  # t,a
F = np.sum(F, axis=1) / na  # Sum over atoms
```

This combines several mathematical steps:
- Calculates phase factors $e^{i\mathbf{k}\cdot\mathbf{r}_j^0}$
- Multiplies by velocities
- Performs the spatial sum $\sum_{j=1}^N$
- Normalizes by number of atoms

### 4. Temporal Fourier Transform
```python
# Perform temporal Fourier transform
integrated = np.fft.fft(F)[:nt2]  # Transform to frequency space
```

This implements the temporal Fourier transform $\mathcal{F}[F(t)]$, converting from time to frequency domain.

## Interpretation

The resulting SED $\Phi(\mathbf{k},\omega)$ represents the energy distribution across:
- Wave vectors $\mathbf{k}$ (related to wavelength)
- Frequencies $\omega$ (related to phonon energy)

### Complete SED Implementation

```python
def SED(avg, velocities, p_xyz, v_xyz, a, z_range=None, nk=100, 
        bs='', perAtom=False, ks='', keepComplex=False):
    """
    Calculate Spectral Energy Density for phonon analysis.
    
    Args:
        avg (numpy.ndarray): Average positions [atoms, xyz]
        velocities (numpy.ndarray): Velocities [timesteps, atoms, xyz]
        p_xyz: Wave vector direction (0=x, 1=y, 2=z or vector)
        v_xyz: Velocity component to analyze
        a (float): Lattice constant/periodicity
        z_range (tuple): Optional (min_z, max_z) to filter atoms
        nk (int): Number of k-points
        bs (array): Atom indices to include
        perAtom (bool): Calculate per-atom contributions
        ks (array): Custom k-points
        keepComplex (bool): Preserve complex values
        
    Returns:
        tuple: (SED_array, wavevectors, frequencies)
    """
    import numpy as np
    from tqdm import tqdm
    
    # Get dimensions
    nt, na, nax = np.shape(velocities)
    
    # Handle z-range filtering
    if z_range is not None:
        z_min, z_max = z_range
        valid_indices = (avg[:, 2] >= z_min) & (avg[:, 2] <= z_max)
        bs = np.where(valid_indices)[0]
        na = len(bs)
    
    # Set default atom selection if none provided
    if len(bs) == 0:
        bs = np.arange(na)
    else:
        na = len(bs)
    
    # Setup frequency and k-space grids
    nt2 = int(nt / 2)
    if len(ks) == 0:
        ks = np.linspace(0, np.pi/a, nk)
    else:
        nk = len(ks)
    ws = np.fft.fftfreq(nt)[:nt2]
    
    # Initialize output array
    if keepComplex:
        Zs = np.zeros((nt2, nk), dtype=complex)
    else:
        Zs = np.zeros((nt2, nk))
    
    # Project positions onto wave vector direction
    if isinstance(p_xyz, (int, float)):
        xs = avg[bs, p_xyz]
    else:
        p_xyz = np.asarray(p_xyz)
        d = p_xyz[None, :] * np.ones((na, 3))
        xs = np.einsum('ij, ij->i', avg[bs, :], d)
        xs /= np.linalg.norm(p_xyz)
    
    # Project velocities
    if isinstance(v_xyz, (int, float)):
        vs = velocities[:, bs, v_xyz]
    else:
        vflat = np.reshape(velocities[:, bs, :], (nt*na, 3))
        v_xyz = np.asarray(v_xyz)
        d = v_xyz[None, :] * np.ones((nt*na, 3))
        vs = np.einsum('ij, ij->i', vflat, d)
        vs = np.reshape(vs, (nt, na))
        vs /= np.linalg.norm(v_xyz)
    
    # Per-atom analysis
    if perAtom:
        Zs = np.zeros((nt2, nk, na), dtype=complex)
        for j, k in enumerate(tqdm(ks)):
            F = vs[:, :] * np.exp(1j * k * xs[None, :])
            integrated = np.fft.fft(F, axis=0)[:nt2, :]
            Zs[:, j, :] += integrated
        return Zs, ks, ws
    
    # Regular analysis
    for j, k in enumerate(tqdm(ks)):
        F = vs[:, :] * np.exp(1j * k * xs[None, :])
        F = np.sum(F, axis=1) / na
        integrated = np.fft.fft(F)[:nt2]
        if keepComplex:
            Zs[:, j] += integrated
        else:
            Zs[:, j] += np.absolute(integrated)**2
    
    return Zs, ks, ws
```

## 4. Practical SED Analysis

This section bridges theory and practice by showing how to actually perform SED calculations.

### Resolution Considerations

1. **K-space Resolution**
   - Maximum resolution limited by simulation size
   - For N unit cells in a direction, maximum meaningful k-points is N
   - Example: 20 unit cells in x → maximum 20 meaningful k-points
   - Set `nk` ≤ number of unit cells in your chosen direction

2. **Frequency Resolution**
   - Determined by trajectory length and velocity output frequency
   - Longer trajectories = better frequency resolution
   - For frequencies up to f, need trajectory length ≥ 1/f

### Common Analysis Scenarios

1. **Basic Longitudinal Phonons**:
```python
# Analyze longitudinal modes along x-direction
Z_long, k, w = SED(avg_positions, velocities,
                   p_xyz=0,      # x-direction wave vector
                   v_xyz=0,      # x-component velocity
                   a=3.615,      # lattice constant
                   nk=100)       # k-point resolution
```

2. **Transverse Phonons**:
```python
# Analyze transverse modes (wave vector in x, motion in y)
Z_trans, k, w = SED(avg_positions, velocities,
                    p_xyz=0,     # x-direction wave vector
                    v_xyz=1,     # y-component velocity
                    a=3.615,
                    nk=100)
```

3. **Region-Specific Analysis**:
```python
# Analyze only atoms in a specific z-range
Z_surface, k, w = SED(avg_positions, velocities,
                      p_xyz=0,
                      v_xyz=0,
                      a=3.615,
                      z_range=(0, 20))  # only atoms in first 20 Å
```

4. **Custom Directions**:
```python
# Analyze along [110] direction
Z_110, k, w = SED(avg_positions, velocities,
                  p_xyz=[1,1,0],  # direction vector
                  v_xyz=[1,1,0],  # same direction for longitudinal
                  a=3.615*np.sqrt(2),  # adjusted periodicity
                  nk=100)
```

## 5. Visualization

This section demonstrates how to create clear and informative visualizations of your phonon analysis results.

### Basic Dispersion Plot
```python
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.imshow(np.log10(Z_total),
           extent=[k[0], k[-1], w[0], w[-1]],
           aspect='auto',
           origin='lower',
           interpolation='none')
plt.xlabel('Wave Vector (2π/a)')
plt.ylabel('Frequency (THz)')
plt.colorbar(label='log₁₀(SED)')
plt.show()
```

### Advanced Visualization
```python
# Combined longitudinal and transverse plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))

# Longitudinal modes
im1 = ax1.imshow(np.log10(Z_long),
                 extent=[k[0], k[-1], w[0], w[-1]],
                 aspect='auto',
                 origin='lower',
                 interpolation='none')
ax1.set_title('Longitudinal Modes')

# Transverse modes
im2 = ax2.imshow(np.log10(Z_trans),
                 extent=[k[0], k[-1], w[0], w[-1]],
                 aspect='auto',
                 origin='lower',
                 interpolation='none')
ax2.set_title('Transverse Modes')

# Add colorbars
plt.colorbar(im1, ax=ax1, label='log₁₀(SED)')
plt.colorbar(im2, ax=ax2, label='log₁₀(SED)')

plt.tight_layout()
plt.show()
```

## 6. Complete Analysis Example

This section ties everything together with a full working example that demonstrates the entire workflow from data loading to final visualization.

```python
import numpy as np
import matplotlib.pyplot as plt

# 1. Load trajectory
pos, vel, ts, types = qdump("trajectory.dump")

# 2. Calculate average positions
avg_pos, _ = avgPos(pos, sx=100, sy=100, sz=100)

# 3. Calculate SED for different modes
Z_l, k, w = SED(avg_pos, vel, p_xyz=0, v_xyz=0, a=3.615, nk=100)
Z_t1, _, _ = SED(avg_pos, vel, p_xyz=0, v_xyz=1, a=3.615, nk=100)
Z_t2, _, _ = SED(avg_pos, vel, p_xyz=0, v_xyz=2, a=3.615, nk=100)

# 4. Combine and plot
Z_total = Z_l + Z_t1 + Z_t2

plt.figure(figsize=(8,6))
plt.imshow(np.log10(Z_total),
           extent=[k[0], k[-1], w[0], w[-1]],
           aspect='auto',
           origin='lower',
           interpolation='none')
plt.xlabel('Wave Vector (2π/a)')
plt.ylabel('Frequency (THz)')
plt.colorbar(label='log₁₀(SED)')
plt.title('Total Phonon Spectrum')
plt.show()
```

## 7. Tips for Best Results

This section provides practical tips and troubleshooting guidance for successful implementation.

1. **Data Quality**
   - Ensure sufficient simulation length for good frequency resolution
   - Use appropriate timestep for capturing high-frequency modes
   - Check that velocities are being output frequently enough
2. **Resolution Settings**
   - Remember k-space resolution is limited by system size
   - Balance between resolution and computation time
   - Consider time window size for frequency resolution
3. **Common Issues**
   - Weak or Unexpected signals: Check simulation temperature and equilibration
   - Noisy data: Consider longer trajectories
   - Missing modes: Verify velocity output frequency is frequent enough
   - Poor k-resolution: Check system size in analysis direction
4. **Performance**
   - Use cached `.npy` files for faster reloading
   - Start with lower resolution for quick tests
   - Consider analyzing shorter time windows initially